## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="indy7"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210214-232708', '20210215-041031', '20210215-085110', '20210215-133753', '20210215-184319', '20210216-005455', '20210216-054418', '20210216-104554', '20210216-152114', '20210216-201729']
['20210217-010926', '20210217-063641', '20210217-113319', '20210217-162106', '20210217-205606']
['20210609-031118', '20210609-033403', '20210609-035213', '20210609-040958', '20210609-044224', '20210609-050750', '20210609-053121', '20210609-055625', '20210609-061757', '20210609-114501']
['20210609-121034', '20210609-123459', '20210609-125450', '20210609-131831', '20210609-134121']


In [5]:
data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

data_pairs_train_failmore = []
for dataset in DATASET_TRAIN_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_failmore)))        
        
        
data_pairs_test_failmore = []
for dataset in DATASET_TEST_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_failmore)))        

train set: 10022
test set: 5002
train fail set: 10044
test fail set: 5025


In [6]:
N_max_train = 10000
N_max_test = 5000

print("=== mix data ===")
data_pairs_train_mixed = mix_data_pairs(
    data_pairs_train_filtered, data_pairs_train_failmore, N_max_train)
data_pairs_test_mixed = mix_data_pairs(
    data_pairs_test_filtered, data_pairs_test_failmore, N_max_test)

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]
data_pairs_train_failmore = data_pairs_train_failmore[:N_max_train]
data_pairs_test_failmore = data_pairs_test_failmore[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))
print("failmore data: {} / {}".format(
    len(data_pairs_train_failmore), 
    len(data_pairs_test_failmore)))
print("mixed data: {} / {}".format(
    len(data_pairs_train_mixed), 
    len(data_pairs_test_mixed)))

=== mix data ===
filtered success: 4723
success from failmore: 277
dataset fails: 5000
dataset all: 10000
filtered success: 2323
success from failmore: 177
dataset fails: 2500
dataset all: 5000
filtered data: 10000 / 5000
failmore data: 10000 / 5000
mixed data: 10000 / 5000


## Train

In [7]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

    
def train_script(data_pairs_train, data_pairs_test, robot_model_root,
                 EPOCHS_S=0, EPOCHS_E=15, BATCH_SIZE=16, LOG_STEP=100):
    current_time = get_now()
    logpath = os.path.join(robot_model_root, current_time)
    try_mkdir(logpath)
    train_log_dir = os.path.join(logpath, 'train')
    test_log_dir = os.path.join(logpath, 'test')
    model_log_dir = os.path.join(logpath, 'model_{}/')
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    shutil.copy(os.path.join(
        PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
    print(f'Log path: {logpath}')
    
    N_train = len(data_pairs_train)
    N_test = len(data_pairs_test)
    random.shuffle(data_pairs_test)
    res_list = []
    
    for epoch in range(EPOCHS_S, EPOCHS_E):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        random.shuffle(data_pairs_train)
        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_train:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_test:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        model.save(model_log_dir.format(epoch + 1))

        print("")
        print("=================================================================")
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )
        print("=================================================================")
        print("")
        res_list.append((float(test_accuracy.result())*100, float(test_loss.result())))
    return res_list

## train filtered data

In [ ]:
ROBOT_MODEL_ROOT_DIV = "{}-filtered".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_filtered, data_pairs_test_filtered

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_1/assets



Epoch 1, Loss: 0.458346962928772, Accuracy: 80.12999725341797, Test Loss: 0.7007805109024048, Test Accuracy: 80.96955108642578



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_14/assets



Epoch 14, Loss: 0.17208482325077057, Accuracy: 93.37000274658203, Test Loss: 0.4030241072177887, Test Accuracy: 84.61538696289062



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered/20210610-075405/model_15/assets



Epoch 15, Loss: 0.15220625698566437, Accuracy: 94.17499542236328, Test Loss: 0.40623098611831665, Test Accuracy: 86.17788696289062

/home/rnb/Projects/rnb-planning/model/latticized/indy7-filtered acc/loss list:
80.97 %, 0.701 
85.26 %, 0.36 
85.87 %, 0.339 
85.25 %, 0.34 
85.13 %, 0.35 
85.64 %, 0.324 
84.94 %, 0.36 
86.04 %, 0.341 
85.69 %, 0.357 
87.03 %, 0.332 
86.11 %, 0.355 
84.67 %, 0.374 
86.45 %, 0.38 
84.62 %, 0.403 
86.18 %, 0.406 


## failmore data

In [8]:
ROBOT_MODEL_ROOT_DIV = "{}-failmore".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_failmore, data_pairs_test_failmore

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_1/assets



Epoch 1, Loss: 0.23848775029182434, Accuracy: 91.75, Test Loss: 0.12286487966775894, Test Accuracy: 96.2940673828125



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_14/assets



Epoch 14, Loss: 0.07948938757181168, Accuracy: 96.61499786376953, Test Loss: 0.07143552601337433, Test Accuracy: 96.72476196289062



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore/20210610-082657/model_15/assets



Epoch 15, Loss: 0.07662347704172134, Accuracy: 96.66000366210938, Test Loss: 0.07159470021724701, Test Accuracy: 96.98517608642578

/home/rnb/Projects/rnb-planning/model/latticized/indy7-failmore acc/loss list:
1: 96.29 %, 0.123 
2: 96.28 %, 0.085 
3: 96.36 %, 0.079 
4: 96.03 %, 0.083 
5: 95.65 %, 0.093 
6: 96.83 %, 0.072 
7: 96.15 %, 0.084 
8: 96.7 %, 0.08 
9: 96.72 %, 0.084 
10: 95.71 %, 0.084 
11: 97.19 %, 0.073 
12: 96.03 %, 0.082 
13: 95.08 %, 0.1 
14: 96.72 %, 0.071 
15: 96.99 %, 0.072 


## balanced data

In [10]:
ROBOT_MODEL_ROOT_DIV = "{}-balanced".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_mixed, data_pairs_test_mixed

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_1/assets



Epoch 1, Loss: 0.25263604521751404, Accuracy: 90.92500305175781, Test Loss: 0.15432198345661163, Test Accuracy: 94.88180541992188



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_14/assets



Epoch 14, Loss: 0.0749693363904953, Accuracy: 97.7550048828125, Test Loss: 0.22361744940280914, Test Accuracy: 94.31089782714844



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_15/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced/20210610-091408/model_15/assets



Epoch 15, Loss: 0.07152915745973587, Accuracy: 97.94999694824219, Test Loss: 0.3263493776321411, Test Accuracy: 94.39102172851562

/home/rnb/Projects/rnb-planning/model/latticized/indy7-balanced acc/loss list:
1: 94.88 %, 0.154 
2: 94.02 %, 0.178 
3: 95.03 %, 0.151 
4: 95.56 %, 0.132 
5: 95.48 %, 0.142 
6: 93.47 %, 0.183 
7: 94.63 %, 0.167 
8: 95.3 %, 0.144 
9: 95.76 %, 0.139 
10: 93.48 %, 0.188 
11: 94.75 %, 0.18 
12: 95.21 %, 0.151 
13: 94.78 %, 0.159 
14: 94.31 %, 0.224 
15: 94.39 %, 0.326 


# Train results - Indy7

#### filtered: 20210610-075405
```
1: 80.97 %, 0.701 
2: 85.26 %, 0.36 
3: 85.87 %, 0.339 
4: 85.25 %, 0.34 
5: 85.13 %, 0.35 
6: 85.64 %, 0.324 
7: 84.94 %, 0.36 
8: 86.04 %, 0.341 
9: 85.69 %, 0.357 
10: 87.03 %, 0.332 
11: 86.11 %, 0.355 
12: 84.67 %, 0.374 
13: 86.45 %, 0.38 
14: 84.62 %, 0.403 
15: 86.18 %, 0.406 
```

#### failmore: 20210610-082657
```
1: 96.29 %, 0.123 
2: 96.28 %, 0.085 
3: 96.36 %, 0.079 
4: 96.03 %, 0.083 
5: 95.65 %, 0.093 
6: 96.83 %, 0.072 
7: 96.15 %, 0.084 
8: 96.7 %, 0.08 
9: 96.72 %, 0.084 
10: 95.71 %, 0.084 
11: 97.19 %, 0.073 
12: 96.03 %, 0.082 
13: 95.08 %, 0.1 
14: 96.72 %, 0.071 
15: 96.99 %, 0.072 
```

#### balanced: 20210610-091408
```
1: 94.88 %, 0.154 
2: 94.02 %, 0.178 
3: 95.03 %, 0.151 
4: 95.56 %, 0.132 
5: 95.48 %, 0.142 
6: 93.47 %, 0.183 
7: 94.63 %, 0.167 
8: 95.3 %, 0.144 
9: 95.76 %, 0.139 
10: 93.48 %, 0.188 
11: 94.75 %, 0.18 
12: 95.21 %, 0.151 
13: 94.78 %, 0.159 
14: 94.31 %, 0.224 
15: 94.39 %, 0.326 
```

## New train 2021-06
* **[IMPORTANT] 이전에 Tej 사용 틀린 점 보완 했음** 

### Panda
* 20210606-080139 : first balanced train trial, 96.6% 달성 / 이전 데이터와 collision boundary shape 차이일 수 있음
* 20210606-082302 : 이전 데이터와 전부 섞어서 collision boundary shape 차이 영향 확인 - 87% 가량, 영향 확실, 재수집 필요
* 20210606-191455 : 새로 모았어도 분리 학습 하니 시작부터 99%, 문제 있음
* 20210606-194148 : 새로 모은거 분리 학습, 테스트만 순수 기존 데이터로 하니 59%, 문제 확실
* 20210606-201525 : 새 데이터만 학습, 96%? 하지만 fail bias 돼 있음.
* 20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

### Indy7
* 20210607-224401 : 새 데이터만 학습, test set은 bias 제거함 ep12 79.5% 수준
* 20210610-052216 : 조인트 리밋 해제한 데이터만 학습, test set은 bias 제거, ep12 92% 수준

## Load & test

20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

In [8]:
def get_tf_functions(model_root, model_folder, epoch_folder):
    # last_model = '20210606-204309' # panda
    # last_model = '20210610-052216' # indy
    # last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
    logpath = os.path.join(model_root, model_folder)

    # epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
    # print(epochlist)
    # last_save = epochlist[-1]
    model_log_dir = os.path.join(logpath, epoch_folder)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    return model, inference, calc_loss

def test_script(data_pairs_test, model_root, model_folder, epoch_folder):
    BATCH_SIZE = 1
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()
    
    model, inference, calc_loss= get_tf_functions(model_root, model_folder, epoch_folder)

    grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
    res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])


    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]

    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)

    TP_FN_FP_TN = (np.sum(np.logical_and(res_list, label_list)), np.sum(np.logical_and(np.logical_not(res_list), label_list)), 
                   np.sum(np.logical_and(res_list, np.logical_not(label_list))), np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list))))
        
    return label_list, res_list

# Test all

In [9]:
print("========= Filtered =========")
model_foler, epoch_folder = '20210610-075405', 'model_10'
filtered_res = []
filtered_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))

print("========= Failmore =========")
model_foler, epoch_folder = '20210610-082657', 'model_11'
failmore_res = []
failmore_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))

print("========= Balanced =========")
model_foler, epoch_folder = '20210610-091408', 'model_9'
balanced_res = []
balanced_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))

print("========= Filtered =========")
print("filtered:")
print_confusion_mat(*filtered_res[0])
print("failmore:")
print_confusion_mat(*filtered_res[1])
print("balanced:")
print_confusion_mat(*filtered_res[2])
print("========================\n")

print("========= Failmore =========")
print("filtered:")
print_confusion_mat(*failmore_res[0])
print("failmore:")
print_confusion_mat(*failmore_res[1])
print("balanced:")
print_confusion_mat(*failmore_res[2])
print("========================\n")

print("========= Balanced =========")
print("filtered:")
print_confusion_mat(*balanced_res[0])
print("failmore:")
print_confusion_mat(*balanced_res[1])
print("balanced:")
print_confusion_mat(*balanced_res[2])
print("========================\n")

========= Filtered =========


test step - 5000/5000        
Test Loss: 0.331659734249115 
Test Accuracy: 86.74 
TP / FN / ACC: 2042, 281, 87.9
FP / TN / ACC: 382, 2295, 85.73
PACC / NACC / TACC: 84.24, 89.09, 86.74


inference: 	133612.0 ms/5000 = 26.722 ms (11.807/34.71)



test step - 5000/5000        
Test Loss: 0.7455330491065979 
Test Accuracy: 71.41999999999999 
TP / FN / ACC: 220, 8, 96.49
FP / TN / ACC: 1421, 3351, 70.22
PACC / NACC / TACC: 13.41, 99.76, 71.42


inference: 	134013.0 ms/5000 = 26.803 ms (11.775/34.24)



test step - 5000/5000        
Test Loss: 0.5408838987350464 
Test Accuracy: 79.67999999999999 
TP / FN / ACC: 2214, 286, 88.56
FP / TN / ACC: 730, 1770, 70.8
PACC / NACC / TACC: 75.2, 86.09, 79.68


inference: 	133252.0 ms/5000 = 26.65 ms (11.208/32.908)

========= Failmore =========


test step - 5000/5000        
Test Loss: 0.6758021712303162 
Test Accuracy: 72.2 
TP / FN / ACC: 1066, 1257, 45.89
FP / TN / ACC: 133, 2544, 95.03
PACC / NACC / TACC: 88.91, 66.93, 72.2


inference: 	134675.0 ms/5000 = 26.935 ms (12.146/39.798)



test step - 5000/5000        
Test Loss: 0.07306021451950073 
Test Accuracy: 97.18 
TP / FN / ACC: 157, 71, 68.86
FP / TN / ACC: 70, 4702, 98.53
PACC / NACC / TACC: 69.16, 98.51, 97.18


inference: 	133043.0 ms/5000 = 26.609 ms (12.058/50.674)



test step - 5000/5000        
Test Loss: 0.5725482702255249 
Test Accuracy: 73.04 
TP / FN / ACC: 1187, 1313, 47.48
FP / TN / ACC: 35, 2465, 98.6
PACC / NACC / TACC: 97.14, 65.25, 73.04


inference: 	133506.0 ms/5000 = 26.701 ms (12.583/46.678)

========= Balanced =========


test step - 5000/5000        
Test Loss: 1.219055414199829 
Test Accuracy: 59.519999999999996 
TP / FN / ACC: 2277, 46, 98.02
FP / TN / ACC: 1978, 699, 26.11
PACC / NACC / TACC: 53.51, 93.83, 59.52


inference: 	132478.0 ms/5000 = 26.496 ms (11.318/33.922)



test step - 5000/5000        
Test Loss: 0.19391264021396637 
Test Accuracy: 93.56 
TP / FN / ACC: 223, 5, 97.81
FP / TN / ACC: 317, 4455, 93.36
PACC / NACC / TACC: 41.3, 99.89, 93.56


inference: 	133637.0 ms/5000 = 26.727 ms (12.326/34.092)



test step - 5000/5000        
Test Loss: 0.13875547051429749 
Test Accuracy: 95.72 
TP / FN / ACC: 2450, 50, 98.0
FP / TN / ACC: 164, 2336, 93.44
PACC / NACC / TACC: 93.73, 97.9, 95.72


inference: 	134969.0 ms/5000 = 26.994 ms (12.649/33.604)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2042 	 	 281 	 	 87.90%
GN 	 382 	 	 2295 	 85.73%
AL 	 84.24% 	 89.09% 	 86.74%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 220 	 	 8 	 	 96.49%
GN 	 1421 	 	 3351 	 70.22%
AL 	 13.41% 	 99.76% 	 71.42%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2214 	 	 286 	 	 88.56%
GN 	 730 	 	 1770 	 70.80%
AL 	 75.20% 	 86.09% 	 79.68%

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1066 	 	 1257 	 	 45.89%
GN 	 133 	 	 2544 	 95.03%
AL 	 88.91% 	 66.93% 	 72.20%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 157 	 	 71 	 	 68.86%
GN 	 70 	 	 4702 	 98.53%
AL 	 69.16% 	 98.51% 	 97.18%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1187 	 	 1313 	 	 47.48%
GN 	 35 	 	 2465 	 98.60%
AL 	 97.14% 	 65.25% 	 73.04

### Test result - indy
```
inference: 	134969.0 ms/5000 = 26.994 ms (12.649/33.604)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2042 	 	 281 	 	 87.90%
GN 	 382 	 	 2295 	 85.73%
AL 	 84.24% 	 89.09% 	 86.74%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 220 	 	 8 	 	 96.49%
GN 	 1421 	 	 3351 	 70.22%
AL 	 13.41% 	 99.76% 	 71.42%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2214 	 	 286 	 	 88.56%
GN 	 730 	 	 1770 	 70.80%
AL 	 75.20% 	 86.09% 	 79.68%
========================

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1066 	 	 1257 	 	 45.89%
GN 	 133 	 	 2544 	 95.03%
AL 	 88.91% 	 66.93% 	 72.20%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 157 	 	 71 	 	 68.86%
GN 	 70 	 	 4702 	 98.53%
AL 	 69.16% 	 98.51% 	 97.18%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1187 	 	 1313 	 	 47.48%
GN 	 35 	 	 2465 	 98.60%
AL 	 97.14% 	 65.25% 	 73.04%
========================

========= Balanced =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2277 	 	 46 	 	 98.02%
GN 	 1978 	 	 699 	 26.11%
AL 	 53.51% 	 93.83% 	 59.52%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 223 	 	 5 	 	 97.81%
GN 	 317 	 	 4455 	 93.36%
AL 	 41.30% 	 99.89% 	 93.56%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2450 	 	 50 	 	 98.00%
GN 	 164 	 	 2336 	 93.44%
AL 	 93.73% 	 97.90% 	 95.72%
========================
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()

## save gridded data

In [ ]:
data_pair = data_pairs_test[0]
grasp_data = load_pickle(data_pair[0])
arm_data = load_pickle(data_pair[1])
grasp_tar_idx = grasp_data[b'tar']
grasp_tool_idx = grasp_data[b'tool']
arm_tar_idx = arm_data[b'tar']
Tee = grasp_data[b'T_end_effector']
Tej = grasp_data[b'T_end_joint']
Tref_base = grasp_data[b'Tref_base']
reach_lb = grasp_data[b'reach']
retrieve_lb = grasp_data[b'retrieve']
r, th, h = cart2cyl(*Tee[:3,3])
r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
r_class = div_r(r_ej)
h_class = div_h(h_ej)
r_mask = np.zeros(RH_MASK_SIZE)
r_mask[r_class*RH_MASK_STEP:r_class*RH_MASK_STEP+RH_MASK_STEP] = 1
h_mask = np.zeros(RH_MASK_SIZE)
h_mask[h_class*RH_MASK_STEP:h_class*RH_MASK_STEP+RH_MASK_STEP] = 1
rh_mask = np.concatenate([r_mask, h_mask])
# r_ej_list.append(r_ej)
# h_ej_list.append(h_ej)
# reach_lb_list.append(reach_lb)
Tref = SE3(Rot_axis(3, th), Tee[:3,3])
grasp_tool_img = np.zeros(GRASP_SHAPE)
grasp_tar_img = np.zeros(GRASP_SHAPE)
grasp_tool_img[np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)] = 1
grasp_tar_img[np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)] = 1
arm_img = np.zeros(ARM_SHAPE+(1,))
arm_img[np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)] = 1
grasp_img = np.stack([grasp_tool_img, grasp_tar_img], axis=-1)

In [ ]:
# save_json("tar_arm.json", np.array(np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)).transpose())

In [ ]:
# save_json("tool_effector.json", np.array(np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)).transpose())
# save_json("tar_effector.json", np.array(np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)).transpose())

## visualize r, h class distribution

In [ ]:
import matplotlib.pyplot as plt
r_ej_list = []
h_ej_list = []
for data_pair in data_pairs_train:
    grasp_data = load_pickle(data_pair[0])
    arm_data = load_pickle(data_pair[1])
    grasp_obj_idx = grasp_data[b'obj']
    grasp_tar_idx = grasp_data[b'tar']
    grasp_tool_idx = grasp_data[b'tool']
    arm_tar_idx = arm_data[b'tar']
    Tee = grasp_data[b'T_end_effector']
    Tej = grasp_data[b'T_end_joint']
    Tref_base = grasp_data[b'Tref_base']
    reach_lb = grasp_data[b'reach']
    retrieve_lb = grasp_data[b'retrieve']
    r, th, h = cart2cyl(*Tee[:3,3])
    r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
    r_ej_list.append(r_ej)
    h_ej_list.append(h_ej)

In [ ]:
def div_r(r):
    return floor(sigmoid((r)/0.1-7)*8)

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-5)*8)

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:

plt.plot(sorted(np.floor(np.divide(r_ej_list,0.1))), '.')
plt.plot(sorted(np.floor(np.divide(h_ej_list,0.1))), '.')

In [ ]:

plt.plot(sorted(r_ej_list), '.')
plt.plot(sorted(h_ej_list), '.')

In [ ]:
np.max(r_ej_list)

In [ ]:
1.0 - 0.2

In [ ]:
np.min((np.array(r_ej_list)-0.2))

In [ ]:
np.min(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int))

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
x = div_r_gaussian(sorted(r_ej_list)[5000])

In [ ]:
y = div_h_gaussian(sorted(h_ej_list)[5000])

In [ ]:
plt.figure(figsize=(11,5))
plt.imshow(x[np.newaxis, :], cmap='gray')

In [ ]:
plt.figure(figsize=(16,5))
plt.imshow(y[np.newaxis, :], cmap='gray')

In [ ]:
def div_h_gaussian(h_val):
    return gaussian(h_val, np.arange(-0.5,1.1, 0.05),0.05)

In [ ]:

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-4.5)*8)

In [ ]:
X = np.repeat(np.transpose([div_r_gaussian(r_val) for r_val in sorted(r_ej_list)]), 100, axis=0)

In [ ]:
X = np.repeat(np.transpose([div_h_gaussian(h_val) for h_val in sorted(h_ej_list)]), 100, axis=0)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(X)

In [ ]:
min(r_ej_list), max(r_ej_list)

In [ ]:
min(h_ej_list), max(h_ej_list)

In [ ]:
plt.plot(sorted(np.clip(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int), 0, 15)))

In [ ]:
Ndat = len(data_pairs_train)
sorted(r_ej_list)